In [2]:
import pandas as pd
import os

# Extracting RF and LST DATA
## Matching Latitude-Longtitude to Sub-District
### Climatic Data

In [3]:
#df_big_area_nakhon_krabi = pd.read_csv(os.path.join('Data','Climatic Data','Climatic Data Result','big_Nakhon_Krabi_LST.csv'))
df_big_area_nakhon_krabi = pd.read_csv(os.path.join('Data','Climatic Data','Climatic Data Result','big_Nakhon_Krabi_Rainfall.csv'))
df_big_area_nakhon_krabi['addrcode'] = df_big_area_nakhon_krabi['addrcode'].astype(str)
df_big_area_nakhon_krabi.head()

,addrcode
0,801904
1,800905
2,801701
3,800611
4,801204


In [4]:
df_big_area_bangkok = pd.read_csv(os.path.join('Data','Climatic Data','Climatic Data Result','big_Bangkok_LST.csv'))
#df_big_area_bangkok = pd.read_csv(os.path.join('Data','Climatic Data','Climatic Data Result','big_Bangkok_Rainfall.csv'))
df_big_area_bangkok['addrcode'] = df_big_area_bangkok['addrcode'].astype(str)
df_big_area_bangkok.head()

,addrcode
0,100508
1,100112
2,104601
3,104101
4,100302


In [5]:
df_point_bangkok = pd.read_csv(os.path.join('Data','Climatic Data','Coordinate','BMA_LT_LST.csv'))
#df_point_bangkok = pd.read_csv(os.path.join('Data','Climatic Data','Coordinate','BMA_LT_Rainfall.csv'))

In [ ]:
df_point_bangkok['X'] = df_point_bangkok['X'].astype(str)
df_point_bangkok['Y'] = df_point_bangkok['Y'].astype(str)
df_point_bangkok['X'] = df_point_bangkok['X'].str[:7]
df_point_bangkok['Y'] = df_point_bangkok['Y'].str[:6]
df_point_bangkok = df_point_bangkok.drop('BMA_CODE', axis = 1)
df_point_bangkok.columns = ['addrcode','X','Y']
df_point_bangkok['addrcode'] = df_point_bangkok['addrcode'].astype(str)
df_point_bangkok.head()

In [ ]:
df_point_nakhon_krabi = pd.read_csv(os.path.join('Data','Climatic Data Result','XY_Coordinate_Nakhon_Krabi.csv'))

In [ ]:
df_point_nakhon_krabi.head()

In [ ]:
df_point_nakhon_krabi['X'] = df_point_nakhon_krabi['X'].astype(str)
df_point_nakhon_krabi['Y'] = df_point_nakhon_krabi['Y'].astype(str)
df_point_nakhon_krabi['X'] = df_point_nakhon_krabi['X'].str[:7]
df_point_nakhon_krabi['Y'] = df_point_nakhon_krabi['Y'].str[:7]
df_point_nakhon_krabi['addrcode'] = df_point_nakhon_krabi['addrcode'].astype(str)
df_point_nakhon_krabi.head()

### Bangkok LST & Rainfall

In [ ]:
from tqdm import tqdm
from os import listdir
path = '/Users/boonpakorn/Documents/GitHub/dengue_data_statistics/Data/Climatic Data/Climatic Data Result/LST Result/'
filename = []
filelist= listdir(path)
for f in tqdm(filelist):
    if f.endswith('Bangkok.csv'):
        filename.append(f)

In [ ]:
len(filename)

In [ ]:
## === Small Area ===
for filename in filename:
    if filename.startswith('Small'):
        df_temp = pd.read_csv(os.path.join(path+filename))
        df_temp['X'] = df_temp['X'].astype(str)
        df_temp['Y'] = df_temp['Y'].astype(str)
        df_temp['X'] = df_temp['X'].str[:7]
        df_temp['Y'] = df_temp['Y'].str[:6]
        df_temp.head()

        df_result = pd.merge(df_temp,df_point_bangkok,how = 'inner', on = ['X','Y'])
        df_result = df_result.drop(['POINT_BANG','X','Y'],axis = 1)
        df_result['MOD11C2_20'] = df_result['MOD11C2_20']*0.02-273.15
        df_result['MOD11C2_20'] = df_result['MOD11C2_20'].round(2)
        df_result = df_result[['addrcode','MOD11C2_20']]
        df_result.columns = ['addrcode','temperature']
        df_result['temperature'].loc[df_result['temperature'] <0 ] = 0

#         df_result = pd.merge(df_temp,df_point_bangkok,how = 'inner', on = ['X','Y'])
#         df_result = df_result.drop(['POINT_BANG','X','Y'],axis = 1)
#         df_result.columns = ['rainfall','addrcode']
#         df_result = df_result[['addrcode','rainfall']]
#         df_result['rainfall'] = df_result['rainfall'].round(2)
#         df_result['rainfall'].loc[df_result['rainfall'] < 0] = None
#         df_result = df_result[['addrcode','rainfall']]

        df_result.to_csv(path+'Code_'+filename, encoding = 'utf-8') 
    else:
        continue

In [ ]:
## === Big Area ===
for filename in filename:
    
    if filename.startswith('MOD'):
        df_temp = pd.read_csv(os.path.join(path+filename))
        df_temp = df_temp.drop(['ZONE_CODE','COUNT','AREA','MIN','MAX','RANGE','STD','SUM','VARIETY','MAJORITY','MINORITY','MEDIAN'], axis = 1)
        df_temp['SCODE_BMA'] = df_temp['SCODE_BMA'].astype(str)
        df_temp['MEAN'].loc[df_temp['MEAN'] > 7500 ] = df_temp['MEAN']*0.02-273.15
        df_temp['MEAN'] = df_temp['MEAN'].round(2)
        df_temp['MEAN'].loc[df_temp['MEAN'] <0 ] = 0
        df_temp.columns = ['addrcode','temperature']
        df_temp.head()
        df_result = pd.merge(df_temp,df_big_area_bangkok,how = 'outer', on ='addrcode')
        df_result = df_result.fillna(0)
        df_result = df_result[['addrcode','temperature']]

#     if filename.startswith('P_Rain'):
#         df_temp = pd.read_csv(os.path.join(path+filename))
#         df_temp = df_temp.drop(['ZONE_CODE','COUNT','AREA','MIN','MAX','RANGE','STD','SUM'], axis = 1)
#         df_temp['MEAN'] = df_temp['MEAN'].round(2)
#         df_temp.columns = ['addrcode','rainfall']
#         df_temp['addrcode'] = df_temp['addrcode'].astype(str)
    
        df_result = pd.merge(df_temp,df_big_area_bangkok,how = 'outer', on ='addrcode')

        df_result.to_csv(path+'Code_'+filename, encoding = 'utf-8')
    else:
        continue


### Nakhon-Krabi LST & Rainfall

In [ ]:
from tqdm import tqdm
from os import listdir
#path = '/Users/boonpakorn/Documents/GitHub/dengue_data_statistics/Data/Climatic Data/Climatic Data Result/LST Result/'
path = '/Users/boonpakorn/Documents/GitHub/dengue_data_statistics/Data/Climatic Data/Climatic Data Result/Rainfall Result/'
filename = []
filelist= listdir(path)
for f in tqdm(filelist):
    if f.endswith('Nakhon_Krabi.csv'):
        filename.append(f)

In [ ]:
len(filename)

In [ ]:
## === Small Area ===
for filename in filename:
    if filename.startswith('Small'):
        df_temp = pd.read_csv(os.path.join(path+filename))
        df_temp['X'] = df_temp['X'].astype(str)
        df_temp['Y'] = df_temp['Y'].astype(str)
        df_temp['X'] = df_temp['X'].str[:7]
        df_temp['Y'] = df_temp['Y'].str[:7]
        df_temp.head()

        #df_result = pd.merge(df_temp,df_point_nakhon_krabi,how = 'inner', on = ['X','Y'])
        #df_result = df_result.drop(['POINT_NAKH','X','Y','TB_EN','OLD_TB_IDN'],axis = 1)
        #df_result['MOD11C2_20'] = df_result['MOD11C2_20']*0.02-273.15
        #df_result['MOD11C2_20'] = df_result['MOD11C2_20'].round(2)
        #df_result['MOD11C2_20'].loc[df_result['MOD11C2_20'] < 0] = None
        #df_result.columns = ['temperature','addrcode']
        #df_result = df_result[['addrcode','temperature']]
        
        df_result = pd.merge(df_temp,df_point_nakhon_krabi,how = 'inner', on = ['X','Y'])
        df_result = df_result.drop(['POINT_NAKH','X','Y','TB_EN','OLD_TB_IDN'],axis = 1)
        df_result.columns = ['rainfall','addrcode']
        df_result = df_result[['addrcode','rainfall']]
        df_result['rainfall'] = df_result['rainfall'].round(2)
        df_result['rainfall'].loc[df_result['rainfall'] < 0] = None
        df_result = df_result[['addrcode','rainfall']]
        
        df_result.to_csv(path+'Code_'+filename, encoding = 'utf-8') 
    else:
        continue

In [ ]:
## === Big Area ===
for filename in filename:
    #if filename.startswith('MOD'):
    if filename.startswith('P_Rain'):
        df_temp = pd.read_csv(os.path.join(path+filename))

        #df_temp = df_temp.drop(['ZONE_CODE','COUNT','AREA','MIN','MAX','RANGE','STD','SUM','VARIETY','MAJORITY','MINORITY','MEDIAN'], axis = 1)
        #df_temp['TB_IDN'] = df_temp['TB_IDN'].astype(str)
        #df_temp.head()
        #df_temp['MEAN'].loc[df_temp['MEAN'] > 7500 ] = df_temp['MEAN']*0.02-273.15
        #df_temp['MEAN'] = df_temp['MEAN'].round(2)
        #df_temp['MEAN'].loc[df_temp['MEAN'] <0 ] = None
        #df_temp.columns = ['addrcode','temperature']
        df_temp = df_temp.drop(['ZONE_CODE','COUNT','AREA','MIN','MAX','RANGE','STD','SUM'], axis = 1)
        df_temp['MEAN'] = df_temp['MEAN'].round(2)
        df_temp.columns = ['addrcode','rainfall']
        df_temp['addrcode'] = df_temp['addrcode'].astype(str)
        
        df_result = pd.merge(df_temp,df_big_area_nakhon_krabi,how = 'outer', on ='addrcode')
        df_result.to_csv(path+'Code_'+filename, encoding = 'utf-8')
            
    else:
        continue



## Combine all files

In [ ]:
des_path = '/Users/boonpakornnonthaleerak/Documents/GitHub/dengue_data_statistics/Data/Climatic Data/Climatic Data Result/LST Result/'
all = []
for filename in filename:
    if filename.startswith('Code_'):
        df_com = pd.read_csv(os.path.join(path+filename))
        if filename.startswith('Code_Small_'):
            if filename.endswith('Bangkok.csv'):
                weekday = filename[:-12][19:] 
                day = weekday[4:]
                year = weekday[:4]
                df_com['Day'] = day
                
#                 weekday = filename[:-12][17:] 
#                 if(len(weekday) == 6): 
#                     year = weekday[2:]
#                     week = weekday[:1]
#                 elif(len(weekday) == 7): 
#                     year = weekday[3:]
#                     week = weekday[:2]
#                 df_com['Week'] = week
                
                
                df_com['Year'] = year
                all.append(df_com)
        #elif filename.startswith('Code_P_Rain'):
        elif filename.startswith('Code_MOD'):
            if filename.endswith('Bangkok.csv'):
                weekday = filename[:-12][13:] 
                year = weekday[:4]
                day = weekday[4:]
                df_com['Day'] = day
                
#                 weekday = filename[:-12][11:] 
#                 if(len(weekday) == 6):    
#                     year = weekday[2:]
#                     week = weekday[:1]
#                 elif(len(weekday) == 7):
#                     year = weekday[3:]
#                     week = weekday[:2]
#                 df_com['Week'] = week
                
                df_com['Year'] = year
                all.append(df_com)

    
big_frame = pd.concat(all, ignore_index=True)
big_frame = big_frame.drop('Unnamed: 0', axis = 1)
big_frame = big_frame[['Day','Year','addrcode','temperature']]
big_frame['temperature'].loc[big_frame['temperature'] == 0 ]  = None

# big_frame = pd.concat(all, ignore_index=True)
# big_frame = big_frame.drop('Unnamed: 0', axis = 1)
# big_frame = big_frame[['Week','Year','addrcode','rainfall']]

big_frame.to_csv('LST_Bangkok.csv', encoding = 'utf-8')


## Concat all Climatic files

In [ ]:
# get data file names
path ='LST Interpolation/Results/'
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)
big_frame = big_frame.drop('Unnamed: 0', axis = 1)
big_frame.to_csv(path +'LST_subdist.csv', encoding = 'utf-8')

In [ ]:
df_lst_dist = pd.read_csv(path+'LST_subdist.csv', index_col =0)
df_lst_dist['addrcode'] = df_lst_dist['addrcode'].astype(str)
df_lst_dist.head()

In [ ]:
df_lst_dist = df_lst_dist.groupby([df_lst_dist['addrcode'].str[:4],df_lst_dist['Year'],df_lst_dist['Week']]).mean().reset_index()
df_lst_dist.head()

In [ ]:
df_lst_dist.to_csv(path+'LST_dist.csv', encoding = 'utf-8')

### Rainfall

In [ ]:
df_rain_dist = pd.read_csv(os.path.join('Data','Climatic Data','Rainfall_subdist.csv'))
df_rain_dist['addrcode'] = df_rain_dist['addrcode'].astype(str)

In [ ]:
df_rain_dist = df_rain_dist.groupby([df_rain_dist['addrcode'].str[:4],df_rain_dist['Year'],df_rain_dist['Week']]).mean().reset_index()
df_rain_dist.head()

In [ ]:
df_rain_dist.to_csv('Data/Climatic Data/Rainfall_dist.csv', encoding = 'utf-8')

## Weekly to Monthly Climatic Data

In [ ]:
df_lst_dist = pd.read_csv(os.path.join('Data','Climatic Data','Rainfall_subdist.csv'))
df_lst_dist['Week'] = df_lst_dist['Week'].astype(str)
df_lst_dist['Year'] = df_lst_dist['Year'].astype(str)
df_lst_dist.head()

In [ ]:
weekyear = []
for i in range(len(df_lst_dist)):
    if len(df_lst_dist['Week'][i]) == 1:
        weekyear.append(df_lst_dist['Year'][i] + '0' + df_lst_dist['Week'][i])
    elif len(df_lst_dist['Week'][i]) == 2:
        weekyear.append(df_lst_dist['Year'][i] + df_lst_dist['Week'][i])

In [ ]:
df_lst_dist['WeekYear'] = weekyear
df_lst_dist['WeekYear'] = df_lst_dist['WeekYear'].astype(int)
df_lst_dist.head()

In [ ]:
df_lst_dist['LastDayWeek'] = pd.to_datetime((df_lst_dist['WeekYear']-1).astype(str) + "6", format="%Y%U%w")
df_lst_dist['Month'] = pd.DatetimeIndex(df_lst_dist['LastDayWeek']).month
df_lst_dist = df_lst_dist.drop(['LastDayWeek','WeekYear'], axis =1)
#df_lst_dist = df_lst_dist[['addrcode','Year','Month','Week','temperature']]
df_lst_dist = df_lst_dist[['addrcode','Year','Month','Week','rainfall']]
df_lst_dist.head()

In [ ]:
df_lst_dist = df_lst_dist.groupby([df_lst_dist['addrcode'],df_lst_dist['Year'],df_lst_dist['Month']]).mean().reset_index()
df_lst_dist.head()

In [ ]:
df_lst_dist = df_lst_dist[df_lst_dist.Year != '2018']

In [ ]:
df_lst_dist.to_csv('Data/Climatic Data/Rainfall_subdist_monthly.csv', encoding = 'utf-8')

## Monthly to Yearly Climatic Data

In [ ]:
df_cli = pd.read_csv(os.path.join('Data','Climatic Data','LST_subdist_monthly.csv'), index_col = 0)
df_cli['addrcode']  = df_cli['addrcode'].astype(str)
df_cli['Month'] = df_cli['Month'].astype(str)
df_cli['Year'] = df_cli['Year'].astype(str)
df_cli.head()

In [ ]:
df_cli = df_cli.groupby([df_cli['addrcode'],df_cli['Year']]).mean().reset_index()
df_cli.head()

In [ ]:
df_cli.to_csv('Data/Climatic Data/LST_subdist_yearly.csv', encoding = 'utf-8')